In [1]:
from pyspark.sql import SparkSession, DataFrame

from pyspark.sql.functions import count, sum, col

In [2]:
spark = SparkSession.builder \
    .appName('Query MySQL database: creditcard_capstone') \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # to see the entire contents of the DataFrame in the console output.


In [3]:
mysql_database_name = 'creditcard_capstone'
mysql_driver = 'com.mysql.jdbc.Driver'
mysql_url = f'jdbc:mysql://localhost:3306/{mysql_database_name}'
mysql_properties = {
                'user': 'root',
                'password': 'password'
}

In [15]:
customer_df = spark.read.jdbc(url=mysql_url, table='customer', properties=mysql_properties)
credit_df = spark.read.jdbc(url=mysql_url, table='credit', properties=mysql_properties)
branch_df = spark.read.jdbc(url=mysql_url, table='branch', properties=mysql_properties)

---

## 2.1 Transaction Details Module

1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.

In [5]:
def get_input_zip_month_year() -> tuple[str, int, int]:
    """
    Prompt the user to enter a zipcode, month, and year, and validate the inputs.

    Returns:
        A tuple containing the validated inputs: (zipcode, month, year)

    Raises:
        ValueError: If any of the inputs are invalid.

    """

    def validate_zipcode(zipcode: str) -> None:
        if len(zipcode) != 5 or not zipcode.isdigit():
            raise ValueError("Invalid zipcode. Please enter a 5-digit numeric value.")

    def validate_month(month: int) -> None:
        if not 1 <= month <= 12:
            raise ValueError("Invalid month. Please enter a numeric value between 1 and 12.")

    def validate_year(year: int) -> None:
        if len(str(year)) != 4 or not str(year).isdigit():
            raise ValueError("Invalid year. Please enter a 4-digit numeric value.")

    while True:
        try:
            zipcode = input("Zipcode: ")
            validate_zipcode(zipcode)
            break
        except ValueError as e:
            print(str(e))

    while True:
        try:
            month = int(input("Month: "))
            validate_month(month)
            break
        except ValueError as e:
            print(str(e))

    while True:
        try:
            year = int(input("Year: "))
            validate_year(year)
            break
        except ValueError as e:
            print(str(e))

    return zipcode, month, year

In [6]:
def transactions_by_zip_month_year() -> None:
    """
    Query credit data by joining with customer data and applying filters.

    Args:
        credit_df (DataFrame): The DataFrame containing the credit data.
        customer_df (DataFrame): The DataFrame containing the customer data.

    Returns:
        DataFrame: The resulting DataFrame after joining and applying filters.

    Raises:
        ValueError: If the required input parameters are not provided.

    """

    input_zip, input_month, input_year = get_input_zip_month_year()
    
    result_df = credit_df.join(customer_df, credit_df.CUST_SSN == customer_df.SSN, 'left') \
                         .where((customer_df.CUST_ZIP == input_zip) & (credit_df.MONTH == input_month) & (credit_df.YEAR == input_year)) \
                         .orderBy(credit_df.DAY)

    result_df.show()

In [7]:
transactions_by_zip_month_year()

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+------+----------------+---------+-------------+-------------------+-----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|  TIMEID|APT_NO|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|         CUST_EMAIL| CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+------+----------------+---------+-------------+-------------------+-----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|        192|4210653387378802|123457047|  2|  

---

2)    Used to display the number and total values of transactions for a given type.

In [8]:
def distinct_options(df: DataFrame, column_name: str) -> list[str]:
    """
    Get the distinct options from a specific column in the DataFrame.

    Args:
        df (DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to fetch distinct options from.

    Returns:
        list[str]: The list of distinct options from the specified column.

    """
    distinct_options = df.select(column_name).distinct().rdd.flatMap(lambda x: x).collect()
    return distinct_options


In [9]:
def get_transaction_type() -> str:
    """
    Prompt the user to enter a valid transaction type and validate the input.

    Returns:
        str: The validated transaction type entered by the user.

    Raises:
        ValueError: If the entered transaction type is not valid.

    """

    valid_transaction_types = distinct_options(df = credit_df, column_name = 'TRANSACTION_TYPE') 
    valid_options = ", ".join(valid_transaction_types)

    while True:
        transaction_type = input(f"Transaction Type ({valid_options}): ")
        if transaction_type in valid_transaction_types:
            break
        print(f"Invalid transaction type. Please enter a valid type from the given options: {valid_options}.")
    
    return transaction_type

In [10]:
def transaction_total_and_no_by_type() -> None:
    """
    Query transaction data based on the user-provided transaction type.

    Returns:
        None

    Raises:
        ValueError: If the input transaction type is not valid.

    """

    # get a valid input from the user
    transaction_type = get_transaction_type() 

    result_df = credit_df.filter(credit_df.TRANSACTION_TYPE == transaction_type) \
                             .select(count('TRANSACTION_ID').alias('Total no of transactions'), \
                                     sum('TRANSACTION_VALUE').alias('Sum of Transaction Values'))

    result_df.show()

In [11]:
transaction_total_and_no_by_type()

+------------------------+-------------------------+
|Total no of transactions|Sum of Transaction Values|
+------------------------+-------------------------+
|                    6635|        338950.1000000002|
+------------------------+-------------------------+



---

3)    Used to display the total number and total values of transactions for branches in a given state.

In [16]:
def get_valid_state() -> str:
    """
    Prompt the user to enter a valid state and validate the input.

    Returns:
        str: The validated state entered by the user.

    Raises:
        ValueError: If the entered state is not valid.

    """

    valid_states = distinct_options(df = branch_df, column_name = 'BRANCH_STATE')
    valid_options = ", ".join(valid_states)

    while True:
        input_state = input(f"State ({valid_options}): ")
        if input_state in valid_states:
            break
        print(f"Invalid state. Please enter a valid state from the given options: {valid_options}.")
    
    return input_state

In [17]:
def transaction_total_and_no_by_branch_on_state ()-> None:
    """
    Process transaction data by querying based on user input of state.

    Args:
        credit_df (DataFrame): The DataFrame containing the credit data.
        customer_df (DataFrame): The DataFrame containing the customer data.

    """

    state = get_valid_state()

    result_df = credit_df.join(branch_df, credit_df.BRANCH_CODE == branch_df.BRANCH_CODE, 'left') \
                         .where(branch_df.BRANCH_STATE == 'MN') \
                         .groupBy(branch_df.BRANCH_CODE) \
                         .agg(count(credit_df.TRANSACTION_ID).alias('Transaction_Count'), sum(credit_df.TRANSACTION_VALUE).alias('Transaction_Sum'))

    result_df.show()

Invalid state. Please enter a valid state from the given options: SC, MN, NJ, VA, KY, MI, WI, CA, CT, MT, NC, MD, IL, WA, MS, AL, IN, OH, IA, PA, NY, TX, GA, MA, FL, AR.
+-----------+-----------------+------------------+
|BRANCH_CODE|Transaction_Count|   Transaction_Sum|
+-----------+-----------------+------------------+
|        124|              372|          18644.89|
|          1|              374|18157.570000000007|
|         20|              381|19350.830000000005|
+-----------+-----------------+------------------+



In [ ]:
transaction_total_and_no_by_branch_on_state()

---

## 2.2 Customer Details Module

1) Used to check the existing account details of a customer.
2) Used to modify the existing account details of a customer.
3) Used to generate a monthly bill for a credit card number for a given month and year.
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.
